<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Artificial Intelligence in Finance

## Convolutional Neural Networks

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Imports

In [ ]:
import os
import math
import numpy as np
import pandas as pd
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
os.environ['PYTHONHASHSEED'] = '0'

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'

In [ ]:
symbol = 'EUR='

In [ ]:
data = pd.DataFrame(pd.read_csv(url, index_col=0,
                                parse_dates=True).dropna()[symbol])

In [ ]:
data.info()

In [ ]:
lags = 5

In [ ]:
features = [symbol, 'r', 'd', 'sma', 'min', 'max', 'mom', 'vol']

In [ ]:
def add_lags(data, symbol, lags, window=20, features=features):
    cols = []
    df = data.copy()
    df.dropna(inplace=True)
    df['r'] = np.log(df / df.shift(1))
    df['sma'] = df[symbol].rolling(window).mean()
    df['min'] = df[symbol].rolling(window).min()
    df['max'] = df[symbol].rolling(window).max()
    df['mom'] = df['r'].rolling(window).mean()
    df['vol'] = df['r'].rolling(window).std()
    df.dropna(inplace=True)
    df['d'] = np.where(df['r'] > 0, 1, 0)
    for f in features:
        for lag in range(1, lags + 1):
            col = f'{f}_lag_{lag}'
            df[col] = df[f].shift(lag)
            cols.append(col)
    df.dropna(inplace=True)
    return df, cols

In [ ]:
data, cols = add_lags(data, symbol, lags, window=20, features=features)

In [ ]:
split = int(len(data) * 0.8)

In [ ]:
train = data.iloc[:split].copy()

In [ ]:
mu, std = train[cols].mean(), train[cols].std()

In [ ]:
train[cols] = (train[cols] - mu) / std

In [ ]:
test = data.iloc[split:].copy()

In [ ]:
test[cols] = (test[cols] - mu) / std

In [ ]:
import random
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten

In [ ]:
def set_seeds(seed=1000):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
set_seeds()
model = Sequential()
model.add(Conv1D(filters=96, kernel_size=5, activation='relu',
                 input_shape=(len(cols), 1)))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
%%time
h = model.fit(np.atleast_3d(train[cols]), train['d'],
          epochs=60, batch_size=48, verbose=False,
          validation_split=0.15, shuffle=False)

In [ ]:
res = pd.DataFrame(h.history)

In [ ]:
res.tail(3)

In [ ]:
res.plot(figsize=(10, 6));

In [ ]:
model.evaluate(np.atleast_3d(test[cols]), test['d'])

In [ ]:
test['p'] = np.where(model.predict(np.atleast_3d(test[cols])) > 0.5, 1, 0)

In [ ]:
test['p'] = np.where(test['p'] > 0, 1, -1)

In [ ]:
test['p'].value_counts()

In [ ]:
(test['p'].diff() != 0).sum()

In [ ]:
test['s'] = test['p'] * test['r']

In [ ]:
ptc = 0.00012 / test[symbol]

In [ ]:
test['s_'] = np.where(test['p'] != 0, test['s'] - ptc, test['s'])

In [ ]:
test[['r', 's', 's_']].sum().apply(np.exp)

In [ ]:
test[['r', 's', 's_']].cumsum().apply(np.exp).plot(figsize=(10, 6));

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">